In [2]:
import os 
import pandas as pd
import numpy as np

path = os.getcwd()

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, BaggingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
le = LabelEncoder()

from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error

import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
path = 'C:\\Users\\sunil\\Projects\\Dockship\\segmind_grand_ai_challenge_2021-dataset'

In [4]:
train = pd.read_csv(path + "\\dataset\\TRAIN.csv")
test = pd.read_csv(path + "\\dataset\\TEST.csv")
ss = pd.read_csv(path + "\\dataset\\sample_submission.csv")
shop = pd.read_csv(path + "\\dataset\\store.csv")

C:\Users\sunil\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
train['Date'] = pd.to_datetime(train['Date'])
test['Date'] = pd.to_datetime(test['Date'])

train['year'] = train['Date'].dt.year
train['month'] = train['Date'].dt.month
train['day'] = train['Date'].dt.day
train['dayofweek'] = train['Date'].dt.dayofweek
train['week'] = train['Date'].dt.week

test['year'] = test['Date'].dt.year
test['month'] = test['Date'].dt.month
test['day'] = test['Date'].dt.day
test['dayofweek'] = test['Date'].dt.dayofweek
test['week'] = test['Date'].dt.week

train['StateHoliday'].replace({0 : 0, '0':0, 'a':1, 'b':2, 'c':3}, inplace = True)
test['StateHoliday'].replace({0 : 0, '0':0, 'a':1, 'b':2, 'c':3}, inplace = True)

In [6]:
# Merging

store_type = dict(zip(shop['Store'], shop['StoreType']))
store_ass = dict(zip(shop['Store'], shop['Assortment']))
store_com = dict(zip(shop['Store'], shop['CompetitionDistance']))
store_month = dict(zip(shop['Store'], shop['CompetitionOpenSinceMonth']))
store_year = dict(zip(shop['Store'], shop['CompetitionOpenSinceYear']))
store_p2 = dict(zip(shop['Store'], shop['Promo2']))
store_pweek = dict(zip(shop['Store'], shop['Promo2SinceWeek']))
store_pyear = dict(zip(shop['Store'], shop['Promo2SinceYear']))
store_pi = dict(zip(shop['Store'], shop['PromoInterval']))

train['Store_Type'] = train['Store'].map(store_type)
train['Store_Assortment'] = train['Store'].map(store_ass)
train['Store_distance'] = train['Store'].map(store_com)
train['Store_CompetitionOpenSinceMonth'] = train['Store'].map(store_month)
train['Store_CompetitionOpenSinceYear'] = train['Store'].map(store_year)
train['Store_Promo2'] = train['Store'].map(store_p2)
train['Store_Promo2SinceWeek'] = train['Store'].map(store_pweek)
train['Store_Promo2SinceYear'] = train['Store'].map(store_pyear)
train['Store_PromoInterval'] = train['Store'].map(store_pi)


test['Store_Type'] = test['Store'].map(store_type)
test['Store_Assortment'] = test['Store'].map(store_ass)
test['Store_distance'] = test['Store'].map(store_com)
test['Store_CompetitionOpenSinceMonth'] = test['Store'].map(store_month)
test['Store_CompetitionOpenSinceYear'] = test['Store'].map(store_year)
test['Store_Promo2'] = test['Store'].map(store_p2)
test['Store_Promo2SinceWeek'] = test['Store'].map(store_pweek)
test['Store_Promo2SinceYear'] = test['Store'].map(store_pyear)
test['Store_PromoInterval'] = test['Store'].map(store_pi)


train['Store_Assortment'].unique()
train['Store_Assortment'].replace({'a':1, 'b':2, 'c':3}, inplace = True)
test['Store_Assortment'].replace({'a':1, 'b':2, 'c':3}, inplace = True)


a = [0, 'Jan,Apr,Jul,Oct', 'Feb,May,Aug,Nov', 'Mar,Jun,Sept,Dec']
b = [0, 1, 2, 3]

train['Store_PromoInterval'].replace(dict(zip(a, b)), inplace = True)
test['Store_PromoInterval'].replace(dict(zip(a, b)), inplace = True)

train['Store_Type'].replace({'a':1, 'b':2, 'c':3, 'd':4}, inplace = True)
test['Store_Type'].replace({'a':1, 'b':2, 'c':3, 'd':4}, inplace = True)

train.fillna(0, inplace = True)
test.fillna(0, inplace = True)

In [7]:
# Changing Datatype

int8_cols = ['DayOfWeek', 'Open', 'Promo','StateHoliday', 'SchoolHoliday', 
'month', 'day', 'dayofweek','week', 'Store_Type', 'Store_Assortment',
'Store_CompetitionOpenSinceMonth','Store_Promo2', 
'Store_Promo2SinceWeek','Store_PromoInterval']

int16_cols = ['Customers', 'year', 'Store', 'Store_CompetitionOpenSinceYear', 'Store_Promo2SinceYear']

int32_cols = ['Sales', 'Store_distance']

def int8(x):
    train[x] = train[x].astype('int8')
    
    test[x] = test[x].astype('int8')

def int16(x):
    train[x] = train[x].astype('int16')
    if x != 'Customers':
        test[x] = test[x].astype('int16')

def int32(x):
    train[x] = train[x].astype('int32')
    if x != 'Sales':
        test[x] = test[x].astype('int32')

for col in int8_cols:
    int8(col)

for col in int16_cols:
    int16(col)

for col in int32_cols:
    int32(col)

In [8]:
# Dropping Samples with Open=0

index = train[ train['Open'] == 0].index
train.drop(index, inplace = True, axis = 0)

In [9]:
train['Sales'] = np.sqrt(train['Sales'])

In [10]:
np.sqrt(0)

0.0

In [11]:
target = 'Sales'
date = 'Date'
customer = 'Customers'

cat_cols = ['Store', 'DayOfWeek', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'year', 'month', 'day', 'dayofweek',
       'week', 'Store_Type', 'Store_Assortment',
       'Store_CompetitionOpenSinceMonth', 'Store_CompetitionOpenSinceYear',
       'Store_Promo2', 'Store_Promo2SinceWeek', 'Store_Promo2SinceYear',
       'Store_PromoInterval']

features = [col for col in train.columns if col not in [target, date, customer]]

In [12]:
trn, val = train_test_split(train, test_size = 0.2, random_state = 1999)

##### Input for model
X_trn, X_val = trn[features], val[features]

##### Target column
y_trn, y_val = trn[target], val[target]

##### Features for test data that we will be predicting
X_test = test[features]

%%time

lr = LGBMRegressor(random_state=1)

lr.fit(X_trn, y_trn)

preds = lr.predict(X_val)
preds = np.abs(preds)

error = np.sqrt(mean_squared_error(y_val, preds))

print(f'mean_squared_log_error is : {error}')
#17**

In [13]:
%%time

lr = XGBRegressor(random_state=1)

lr.fit(X_trn, y_trn)

preds = lr.predict(X_val)
preds = np.abs(preds)

error = np.sqrt(mean_squared_error((y_val)**2, (preds)**2))

print(f'mean_squared_log_error is : {error}')
#1125

mean_squared_log_error is : 1100.5007253984288
Wall time: 8.93 s


In [14]:
%%time
params = {'max_depth': 14, 'n_estimators': 1059, 'learning_rate': 0.7436704297351775, 
 'colsample_bytree': 0.5822107008573151, 'reg_alpha': 54.53382998139, 'reg_lambda': 42.423114453956586}

lr = XGBRegressor(random_state=1, **params)

lr.fit(X_trn, y_trn, )

preds = lr.predict(X_val)
preds = np.abs(preds)

error = np.sqrt(mean_squared_error((y_val)**2, (preds)**2))

print(f'mean_squared_log_error is : {error}')
#1125

mean_squared_log_error is : 687.2267453865767
Wall time: 3min 20s


%%time
params = {'max_depth': 14, 'n_estimators': 1059, 'learning_rate': 0.7436704297351775, 
 'colsample_bytree': 0.5822107008573151, 'reg_alpha': 54.53382998139, 'reg_lambda': 42.423114453956586}

lr = XGBRegressor(random_state=1, **params)

lr.fit(X_trn, y_trn, eval_set = [(X_val, y_val)], early_stopping_rounds = 80)

preds = lr.predict(X_val)
preds = np.abs(preds)

error = np.sqrt(mean_squared_error((y_val)**2, (preds)**2))

print(f'mean_squared_log_error is : {error}')
#1125

%%time

lr = CatBoostRegressor(random_state=1)

lr.fit(X_trn, y_trn)

preds = lr.predict(X_val)
preds = np.abs(preds)

error = np.sqrt(mean_squared_error(y_val, preds))

print(f'mean_squared_log_error is : {error}')
#1047

%%time
cat_cols = ['Store', 'DayOfWeek', 'Promo', 'dayofweek']

lr = CatBoostRegressor(random_state=1, cat_features=cat_cols)

lr.fit(X_trn, y_trn)

preds = lr.predict(X_val)
preds = np.abs(preds)

error = np.sqrt(mean_squared_error(y_val, preds))

print(f'mean_squared_log_error is : {error}')
###### Without Cat cols: 1047
#840
#847 cat_cols 2

%%time

lr = CatBoostRegressor(random_state=1, cat_features=cat_cols)

lr.fit(train[features], train[target])

preds = lr.predict(X_test)
preds = np.abs(preds)
#cat with4 cat features lb score 1041

%%time

cat_cols = ['Store', 'Promo']
lr = CatBoostRegressor(random_state=1, cat_features=cat_cols, n_estimators=5000)

lr.fit(train[features], train[target], eval_set = [(X_val, y_val)], early_stopping_rounds = 80)

preds = lr.predict(X_test)
preds = np.abs(preds)
# With eval set and no cat_cols: lb score 1093

In [20]:
%%time
params = {'max_depth': 14, 'n_estimators': 1059, 'learning_rate': 0.7436704297351775, 
 'colsample_bytree': 0.5822107008573151, 'reg_alpha': 54.53382998139, 'reg_lambda': 42.423114453956586}
lr = XGBRegressor(random_state=1, n_jobs = -1, **params)

lr.fit(train[features], train[target])

preds = lr.predict(X_test)
preds = np.abs(preds)
# With eval set and no cat_cols: lb score 1093

Wall time: 6min 28s


In [21]:
preds = preds**2
test['Predictions'] = preds
test.loc[ test['Open'] == 0, 'Predictions'] = 0
preds = test['Predictions']

index = [i for i in range(test.shape[0])]

d = list(zip(index, preds))

ss = pd.DataFrame(d, columns = ['index', 'Sales'])

In [22]:
ss.to_csv(path + "\\baseline_cat.csv", index = False)

In [282]:
{'max_depth': 14, 'n_estimators': 1059, 'learning_rate': 0.7436704297351775, 'colsample_bytree': 0.5822107008573151, 'reg_alpha': 54.53382998139, 'reg_lambda': 42.423114453956586}

,index,Sales
0,0,0.000000
1,1,3922.651855
2,2,6902.786133
3,3,3894.173340
4,4,6356.049805
...,...,...
305158,305158,9431.999023
305159,305159,7354.692871
305160,305160,4196.920898
305161,305161,2640.353271


In [23]:
ss

,index,Sales
0,0,0.000000
1,1,3356.283691
2,2,6645.156738
3,3,3700.219971
4,4,7940.783691
...,...,...
305158,305158,11650.657227
305159,305159,7599.108887
305160,305160,5368.822266
305161,305161,2829.407715
